In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, TensorDataset
from torchvision.utils import save_image
import os

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU instead.")
# device = torch.device("cpu")

Using GPU: Tesla V100-PCIE-32GB


In [14]:
import numpy as np
image_size = 64  # Adjust based on your needs

transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # Normalizing for [-1, 1]
])

data_directory = 'tai_chi/'  # Update with the path to your image directory

dataset = datasets.ImageFolder(root=data_directory, transform=transform)
batch_size = 128  # Adjust based on your GPU's memory
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

FileNotFoundError: Found no valid file for the classes class. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim, img_channels, img_size):
        super(Generator, self).__init__()
        self.img_channels = img_channels
        self.img_size = img_size

        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, 256),
            nn.ReLU(True),
            nn.BatchNorm1d(256),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.BatchNorm1d(512),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.BatchNorm1d(1024),
            nn.Linear(1024, img_channels * img_size * img_size),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), self.img_channels, self.img_size, self.img_size)
        return img

class Discriminator(nn.Module):
    def __init__(self, img_channels):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(img_channels, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
latent_dim = 100
img_size = 64
img_channels = 3
batch_size = 128
epochs = 100
learning_rate = 0.00001

# Initialize models
generator = Generator(latent_dim, img_channels, img_size).to(device)
discriminator = Discriminator(img_channels).to(device)

# Loss and optimizers
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate)
optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate*2)

In [ ]:
def generate_and_save_images(model, epoch, device, folder='chi'):
    if not os.path.exists(folder):
        os.makedirs(folder)
    epoch_folder = os.path.join(folder, f"epoch_{epoch}")
    if not os.path.exists(epoch_folder):
        os.makedirs(epoch_folder)
    with torch.no_grad():
        test_noise = torch.randn(64, 100, device=device)
        generated_images = model(test_noise).cpu()
        for i, img in enumerate(generated_images):
            save_image(img, os.path.join(epoch_folder, f"img_{i}.png"), normalize=True)

In [ ]:
for epoch in range(1, epochs+1):
    # print(epoch, "from outside")
    for i, (real_images, _) in enumerate(dataloader):
        real_images = real_images.to(device)
        batch_size = real_images.size(0)
        # Train Discriminator
        discriminator.zero_grad()
        labels = torch.full((batch_size, 1), 1.0, dtype=torch.float, device=device)*0.9
        output = discriminator(real_images)
        loss_real = criterion(output, labels)
        loss_real.backward()

        noise = torch.randn(batch_size, latent_dim, device=device)
        fake_images = generator(noise)
        labels.fill_(0.1)
        output = discriminator(fake_images.detach())
        loss_fake = criterion(output, labels)
        loss_fake.backward()
        optimizer_D.step()

        # Train Generator
        generator.zero_grad()
        labels.fill_(1.)
        output = discriminator(fake_images)
        loss_G = criterion(output, labels)
        loss_G.backward()
        optimizer_G.step()
    print(f"Epoch {epoch}/{epochs} Loss D: {loss_real.item() + loss_fake.item()}, Loss G: {loss_G.item()}")
    # Logging and Saving Images
    if epoch % 10 == 0:
        generate_and_save_images(generator, epoch, device)
        # # Optionally save the model
        # torch.save(generator.state_dict(), f'generator_epoch_{epoch}.pth')
        # torch.save(discriminator.state_dict(), f'discriminator_epoch_{epoch}.pth')